# Customizing Lens

Lens strives to give you sensible defaults and automatically do the proper assessments whenever possible. However, there are many times where you'll want to change, select, or extend the functionality. Lens is intended to be an _extensible framework_ that can accomodate your own analysis plan.

In this document we will describe a number of ways you can customize Lens:
* Setting up CredoModels
* Selecting which assessments to run
* Parameterizing assessments
* Creating new modules and assessments
* Incorporating new metrics

### Find the code
This notebook can be found on [github](https://github.com/credo-ai/credoai_lens/blob/develop/docs/notebooks/lens_customization.ipynb).

In [ ]:
import credoai.lens as cl

from copy import deepcopy

## Setup

We'll set up data and a model to run through some customization options. See `quickstart` for more information on this code.

In [ ]:
# imports for example data and model training
from credoai.data import fetch_creditdefault
from sklearn.ensemble import GradientBoostingClassifier
# data
data = fetch_creditdefault()
df = data['data']
df['target'] = data['target'].astype(int)
# model
model = GradientBoostingClassifier()
X = df.drop(columns=['SEX', 'target'])
y = df['target']
model.fit(X,y)

In [ ]:
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          data=df,
                          sensitive_feature_key='SEX',
                          label_key='target'
                         )
alignment_spec = {'Fairness': {'metrics': ['precision_score']}}

## Selecting Assessments

**Automatic selection of assessments**

Lens has a number of assessments available, each of which works with different kinds of models or datasets. By default, Lens will automatically run every assessment that has its prerequesites met. The prerequesities can be queried by calling the `get_requirements` function on an assessment. These indicate the set of features or functions your model and data must instantiate in order for the assessment to be run. 

These requirements are specific for the model and the data. Each requirement is either a single requirements or a tuple. If a tuple, only one of the requirements within the tuple must be met. For instance, the FairnessAssessment needs *either* `prob_fun` OR `pred_fun`. See `credoai.assessments.credo_assessment.AssessmentRequirements` for more.

In [ ]:
from credoai.assessment import FairnessAssessment
assessment = FairnessAssessment()
assessment.get_requirements()

You can also get the requirements for all assessments.

In [ ]:
from credoai.assessment import get_assessment_requirements
get_assessment_requirements()

**Selecting a subset of assessments**

But what if you don't want all of those assessments? No problem! Just pass the assessments you do want to Lens. 

In [ ]:
# Note: Remember to instantiate the assessment!
selected_assessments = [FairnessAssessment()]
lens = cl.Lens(model=credo_model,
               data=credo_data,
               assessments=selected_assessments,
               spec=alignment_spec)

## Parameterizing Assessments

Now that we can select assessments, how about customizing them? There are two places where assessments can be customized: (1) when their underlying module is initialized and (2) when they are ran (which runs the underlying module).

### Customizing initialization
Below we can see how to customize the initialization of the assessment. We use something we've already seen before - the `alignment spec`! The parameters that can be passed at this stage are the same parameters passed to the assessment's `init_module` function.

In [ ]:
# Note: Remember to instantiate the assessment!
selected_assessments = [FairnessAssessment()]
init_kwargs = deepcopy(alignment_spec)
init_kwargs['Fairness']['metrics'].append('recall_score')
lens = cl.Lens(model=credo_model,
               data=credo_data,
               assessments=selected_assessments,
               spec=init_kwargs)

In [ ]:
# get the fairness results, from the Fairness assessment, run on the validation dataset
lens.run_assessments().get_results()['validation']['Fairness']['fairness']

### Customizing running assessments 

The other way of parameterizing the assessments is by passing arguments to the assessment's `run` function. These kwargs are passed to `lens.run_assessments`, which are, in turn passed to the assessment's initialized module.

For instance, the `Fairness` assessment initializes `mod.Fairness`, whose `run` argument can take a `method` parameter which controls how fairness scores are calculated. The default is "between_groups", but we can change it like so:

In [ ]:
run_kwargs = {'Fairness': {'method': 'to_overall'}}
lens.run_assessments(assessment_kwargs = run_kwargs).get_results()['validation']['Fairness']['fairness']

## Creating New Modules & Assessments

WIP section!

## Module Specific Customization - Custom Metrics for Fairness Base

Each module has different parameterization options, as discused above. The FairnessModule takes a set of metrics to calculate on the model and data. Many metrics are supported out-of-the-box. These metrics can be referenced by string. However, custom metrics can be created as well. Doing so will allow you to calculate any metric that takes in a `y_true` and some kind of prediction

Custom metrics can be incorporated by creating a `Metric` object. `Metrics` are lightweight wrapper classes that defines a few characteristics of the custom function needed by Lens. 

**Example: Confidence Intervals**

We will create custom metrics that reflect the lower and upper 95th percentile confidence bound on the true positive rate.

Confidence intervals are not generically supported. However, they can be derived for metrics derived from a confusion matrix using the `wilson confidence interval`. A convenience function called `confusion_wilson` is supplied which returns an array: [lower, upper] bound for the metric. 

Wrapping the confusion wilson function in a `Metric` allows us to pass it as a metric to the FairnessModule.

In [ ]:
from credoai.metrics.credoai_metrics import confusion_wilson
from credoai.metrics import Metric

# define partial functions for the true positive rate lower bound
def lower_bound_tpr(y_true, y_pred):
    return confusion_wilson(y_true, y_pred, metric='true_positive_rate', confidence=0.95)[0]

# and upper bound
def upper_bound_tpr(y_true, y_pred):
    return confusion_wilson(y_true, y_pred, metric='true_positive_rate', confidence=0.95)[1]

# wrap the functions in fairness functions
lower_metric = Metric(name = 'lower_bound_tpr', 
                      metric_category = "binary_classification",
                      fun = lower_bound_tpr)

upper_metric = Metric(name = 'upper_bound_tpr', 
                      metric_category = "binary_classification",
                      fun = upper_bound_tpr)

In [ ]:
# Note: Remember to instantiate the assessment!
selected_assessments = [FairnessAssessment()]
init_kwargs = deepcopy(alignment_spec)
init_kwargs['Fairness']['metrics'] = [lower_metric, 'tpr', upper_metric]
custom_lens = cl.Lens(model=credo_model,
               data=credo_data,
               assessments=selected_assessments,
               spec=init_kwargs)
custom_lens.run_assessments().get_results()